# Value + error

Use valuation model and error model in order to determine when to trade

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import keras
import seaborn as sns
import matplotlib.pyplot as plt
from keras import layers
import pandas_datareader as pdr
from datetime import datetime
from keras.models import load_model
from keras.utils import CustomObjectScope
from keras.initializers import glorot_uniform
import itertools
from keras.utils import CustomObjectScope
from keras.initializers import glorot_uniform


Using TensorFlow backend.


In [2]:

NUM_INPUT_NEURONS = 64
NUM_OUTPUT_NEURONS = 1
DAY_OFFSET = 5

def from_network(symbol):
    return pdr.get_data_yahoo(symbols=symbol, start=datetime(1900, 1, 1)).sort_values(by=['Date'],ascending=False)

# Create features (only close price for now)
def convert_to_percentage(old, new):
    return (old - new) / old

def convert_labels_to_category(labels): 
    # Simplification - If positive return, 1, else 0
    # return map(lambda arr: 1 if arr[0] > 1 else 0, labels)
    # rounding simpliciation
    return map(lambda arr: map(lambda val: round(val,4),arr), labels)

def convert_to_train(raw_dataset, offset=5):
    dataset = raw_dataset.copy()
    features = []
    labels = []
    for i in range(offset, len(dataset) - NUM_INPUT_NEURONS):

        feature_dataset = dataset[i:i+NUM_INPUT_NEURONS].copy()
        latest_close = feature_dataset['Close'].iloc[0]
        
        features.append(
            feature_dataset['Close']
                .map(lambda current: convert_to_percentage(latest_close, current))
                .tolist()
        )
        labels.append([
            dataset['Close'].iloc[i-1] / latest_close, # 1 day trade
        ])
        
    # Without converting labels the precision is hard to determine accuracy. 
    # Rather than crude 0/1, maybe this can be more sophisticated
    labels = convert_labels_to_category(labels)
    
    return [features,labels]

def split_data(symbol):
    fetched = from_network(symbol)
    converted = convert_to_train(fetched)
    features = converted[0]
    labels = converted[1]
    training = [ # since we decide a model, use all data for training
        features[1:],
        labels[1:]
    ]
    validation = [
        features[5:1000],
        labels[5:1000]
    ]
    prediction = [
        features[:5],
        labels[:5],
    ]
    
    return {
        'symbol': symbol,
        'prediction': prediction,
        'validation': validation,
        'training': training,
    }

    
def combine_all(accum, prep):
    return {   
        'prediction':[
            accum['prediction'][0] + prep['prediction'][0],
            accum['prediction'][1] + prep['prediction'][1],
        ],
        'validation':[
            accum['validation'][0] + prep['validation'][0],
            accum['validation'][1] + prep['validation'][1],
        ],
        'training':[
            accum['training'][0] + prep['training'][0],
            accum['training'][1] + prep['training'][1],
        ],
    }



In [20]:
with CustomObjectScope({'GlorotUniform': glorot_uniform()}):
    error_model = load_model('sector_model_error.h5')
    valuation_model = load_model('sector_model.h5')



In [4]:
prepped_data = map(split_data, [
    'QQQ',
    'SPY',
    'IWM',
    'XLK',
    'XLF',
    'XLE',
    'XLP',
    'XLV',
    'XLY',
    'XLI',
    'XLU',
])


In [5]:
for i in range(len(prepped_data)):
    print prepped_data[i]['symbol']
combined = reduce(combine_all, prepped_data,{
    'prediction':[[],[]],
    'validation':[[],[]],
    'training':[[],[]],
})

#len(converted_feature_set[0][0])
# print len(combined['training'][0])
# train_data = np.array(combined['training'][0])
# train_labels = np.array(combined['training'][1])

print len(combined['validation'][0])
validation_data = np.array(combined['validation'][0])
validation_labels = np.array(combined['validation'][1])

# print len(combined['prediction'][0])
# prediction_data = np.array(combined['prediction'][0])
# prediction_labels = np.array(combined['prediction'][1])


QQQ
SPY
IWM
XLK
XLF
XLE
XLP
XLV
XLY
XLI
XLU
10945


In [47]:
value_predictions = valuation_model.predict(validation_data)
error_data = np.array(map(lambda inputs: list(itertools.chain(inputs[0],inputs[1])), zip(validation_data, value_predictions)))
error_predictions = error_model.predict(error_data)




In [48]:
# value_predictions

In [49]:
# error_predictions

In [50]:
# validation_labels

In [66]:
# signal using error term modifier with valuation
signals = map(lambda arr: (arr[0] - arr[1])[0],zip(value_predictions, error_predictions))
# signals = map(lambda arr: (1 - arr[1])[0],zip(value_predictions, error_predictions))
# signal using valuation only
# signals = map(lambda arr: (arr[0])[0],zip(value_predictions, error_predictions))

trades = map(lambda signal: 1 if round(signal,4) > 1.00 else 0,signals)

In [67]:
# one day return
actual = map(lambda arr: arr[0],validation_labels)

df = pd.DataFrame({

})
df['actual'] = actual
df['signal'] = signals
df['valuation'] = value_predictions.flatten()
df['error'] = error_predictions.flatten()
df['trade'] = trades
df['entry_success'] = df.apply (lambda row: 1 if (row['actual'] > 1.00 and row['trade'] == 1) else 0,axis=1)
df['entry_failure'] = df.apply (lambda row: 1 if (row['actual'] < 1.00 and row['trade'] == 1) else 0,axis=1)
df['avoid_success'] = df.apply (lambda row: 1 if (row['actual'] < 1.00 and row['trade'] == 0) else 0,axis=1)
df['avoid_failure'] = df.apply (lambda row: 1 if (row['actual'] > 1.00 and row['trade'] == 0) else 0,axis=1)

# primitive policy - replace with a policy network which maximizes reward
def label_success (row):
    return 0 if (row['entry_failure'] == 1 or row['entry_failure'] == 1) else 1

success = df.apply (lambda row: label_success (row),axis=1)
df['success'] = success;
df.head(20)

,actual,signal,valuation,error,trade,entry_success,entry_failure,avoid_success,avoid_failure,success
0,0.9876,0.995610,1.000029,0.004419,0,0,0,1,0,1
1,1.0119,1.013628,1.002701,-0.010928,1,1,0,0,0,1
2,1.0065,0.996428,1.002894,0.006466,0,0,0,0,1,1
3,1.0013,0.998038,1.000287,0.002249,0,0,0,0,1,1
4,0.9800,1.005182,1.001204,-0.003978,1,0,1,0,0,0
5,1.0099,0.987769,0.999815,0.012046,0,0,0,0,1,1
6,1.0079,0.999669,1.001262,0.001592,0,0,0,0,1,1
7,0.9998,0.992502,1.000142,0.007641,0,0,0,1,0,1
8,1.0195,1.000939,0.999013,-0.001926,1,1,0,0,0,1
9,0.9912,0.999034,1.001458,0.002423,0,0,0,1,0,1


In [68]:
print df['actual'].corr(df['signal'])
print df['actual'].corr(df['trade'])
print df['actual'].corr(df['entry_success'])
print df['actual'].corr(df['avoid_success'])

0.026523054242380592
0.02560113190830687
0.4313730564806579
-0.4127708136845405


In [69]:
NUM_TEST_SAMPLES = len(validation_data)
print NUM_TEST_SAMPLES

print '\nPrecision' # optimize for this since we can increase discovery, so long as we find enough trades
print sum(df['entry_success']) * 1.00 / (sum(df['entry_success']) + sum(df['entry_failure'])) 

print '\nRecall'
print sum(df['entry_success']) * 1.00 / (sum(df['entry_success']) + sum(df['avoid_failure']))

print '\nAccuracy'
print sum(df['entry_success']) * 1.00 / (NUM_TEST_SAMPLES)

print '\nNon-loss events'
print sum(df['success'])
print sum(df['success']) / (NUM_TEST_SAMPLES * 1.00)

print '\nLose trades'
print sum(df['entry_failure'])
print sum(df['entry_failure']) / (NUM_TEST_SAMPLES * 1.00)

print '\nWin trades'
print sum(df['entry_success'])
print sum(df['entry_success']) / (NUM_TEST_SAMPLES * 1.00)

print '\nMissed opportunities'
print sum(df['avoid_failure'])
print sum(df['avoid_failure']) / (NUM_TEST_SAMPLES * 1.00)

print '\nBullets dodged'
print sum(df['avoid_success'])
print sum(df['avoid_success']) / (NUM_TEST_SAMPLES * 1.00)


10945

Precision
0.545806668912

Recall
0.557639366827

Accuracy
0.296116948378

Non-loss events
8248
0.75358611238

Lose trades
2697
0.24641388762

Win trades
3241
0.296116948378

Missed opportunities
2571
0.234901781635

Bullets dodged
2345
0.2142530836


In [70]:
# prod below this line